## Time ordered interactions
Produces an interaction list from a ```.mymridon``` experiment file and saves it as a csv. You can specify different matchers (interactions with specific antIDs, interactions of specific body parts) and output this into a csv with the time of each interaction.

In [1]:
from datetime import datetime, timedelta

import pandas as pd
import py_fort_myrmidon as fm

In [2]:
def output_focal_interactions(exp, antID_list, start_time, duration_hours):
    """Function to obtain list of interactions between focal ant and other ants that focus on specific body parts (in this case legs)

    Args:
        exp (py_fort_myrmidon.Experiment): py_fort_myrmidon experiment object
        antID_list (list): List of focal ants
        start_time (Time object): Start time object for experiment
        duration_hours (integer): Duration of the experiment in hours
    """
    # Create timestamps
    t_start = datetime(
        start_time[0], start_time[1], start_time[2], start_time[3], start_time[4]
    ).astimezone(tz=None)
    t_end = t_start + timedelta(hours=duration_hours)
    # Create matcher
    focal_IDs = antID_list
    # Create Or matcher to select interactions between head (1) and leg (3)
    focal_interaction_types = fm.Matcher.Or(
        fm.Matcher.InteractionType(1, 3), fm.Matcher.InteractionType(3, 1)
    )
    # Create matchers for each antID, interaction type combination
    focal_interactions = [
        fm.Matcher.And(fm.Matcher.AntID(x), focal_interaction_types) for x in focal_IDs
    ]
    # Create single matcher object by unpacking the list within an Or Matcher
    focal_matches = fm.Matcher.Or(*focal_interactions)
    # Output trajectories
    trajectories, interactions = fm.Query.ComputeAntInteractions(
        exp, start=t_start, end=t_end, matcher=focal_matches
    )
    # Get a list of lists with details of each interaction
    l_interaction = []
    for ia in interactions:
        l_interaction.append(
            list(ia.IDs)
            + [ia.Start, ia.End, ia.Space]
            + [(ia.End - ia.Start).Seconds()]
            + [any((ia.Types[:] == [1, 3]).all(1)), any((ia.Types[:] == [3, 1]).all(1))]
        )
    # This part can be replaced with the alternate code by matthias which may be faster
    # # get antIDs from the interaction object
    # antIDs = [x.IDs for x in interactions]
    # # Unpack the list
    # antID1, antID2 = map(list, zip(*antIDs))
    # # Obtain start times of the interactions
    # start_times = [str(x.Start) for x in interactions]
    # # Obtain duration of interactions
    # duration = [(x.End - x.Start).Seconds() for x in interactions]
    interaction_df = pd.DataFrame(
        l_interaction,
        columns=[
            "antID1",
            "antID2",
            "start",
            "end",
            "space",
            "duration(s)",
            "1-3",
            "3-1",
        ],
    )
    interaction_df.to_csv(
        "focal_ant_interaction_times_{}_{}_{}.csv".format(exp.Name, t_start, t_end),
        index=False,
    )

In [3]:
def interaction_times(myrmidon_exp, t_start, t_end, colony, phase):
    """Function to obtain time ordered list of interactions between all ants in the experiment

    Args:
        myrmidon_exp (py_fort_myrmidon.Experiment): py_fort_myrmidon experiment object
        t_start (datetime): Start time object for experiment
        t_end (datetime): End time object for experiment
        colony (string): Name of the colony
        phase (string): Phase of the experiment
    """
    # Create matcher for head-head interactions
    m = fm.Matcher.InteractionType(1, 1)
    # Get interactions in a specific time window
    _, interactions = fm.Query.ComputeAntInteractions(
        myrmidon_exp, start=t_start, end=t_end, matcher=m
    )
    # Create an output dataframe
    interactions_df = pd.DataFrame(
        [x.Trajectories[0].Trajectory.Ant for x in interactions], columns=["antID1"]
    )
    interactions_df["antID2"] = [x.Trajectories[1].Trajectory.Ant for x in interactions]
    interactions_df["Time"] = [x.Start for x in interactions]
    interactions_df["Duration"] = [(x.End - x.Start).Seconds() for x in interactions]
    interactions_df["Space"] = [
        x.Trajectories[0].Trajectory.Space for x in interactions
    ]

    # Save the dataframe to a csv file
    day_exp = t_start.strftime("%Y%m%d")
    hr_strt = t_start.strftime("%H%M")
    hr_end = t_end.strftime("%H%M")
    out_file_name = (
        f"interaction_times_{colony}_{phase}_{day_exp}_{hr_strt}_{hr_end}.csv"
    )
    interactions_df.to_csv(out_file_name, index=False)


In [4]:
# Create list of colonies repeated 5 times for each colony
colony_list = ["Cfel42", "Cfel1", "Cfel54", "Cfel13", "Cfel55", "Cfel64"]
# colony_list = ["Cfel42", "Cfel1", "Cfel54", "Cfel13", "Cfel55"]
# Multiple each element 5 times
colony_list = [x for x in colony_list for _ in range(16)]

In [5]:
phase_list = [
    "Base1",
    "Base2",
    "Base3",
    "Base4",
    "Base5",
    "PostC",
    "PostR1",
    "PostR2",
    "PostR3",
    "PostR4",
    "PostR5",
    "R1",
    "R2",
    "R3",
    "R4",
    "R5",
] * 6
# Change to 6 if all 6 colonies are used

In [ ]:
# Myrmidon experiment object list
cfel42_exp = fm.Experiment.Open(
    "/media/ebiag/Ebi-2/Woundcare Experiment1/Cfell_wound_col42.myrmidon"
)
cfel1_exp = fm.Experiment.Open(
    "/media/ebiag/Ebi-2/Woundcare Experiment2/woundcare_cfell1_T2.myrmidon"
)
cfel54_exp = fm.Experiment.Open(
    "/media/ebiag/Ebi-2/Woundcare Experiment3/woundcare_cfell54_T3.myrmidon"
)
cfel13_exp = fm.Experiment.Open(
    "/media/ebiag/Ebi-3/InfectionExp_Cfel13/InfectionExp_Cfel13.myrmidon"
)
cfel55_exp = fm.Experiment.Open(
    "/media/ebiag/Ebi-1/InfectionExp_Cfel55/InfectionExpCol55.myrmidon"
)
cfel64_exp = fm.Experiment.Open(
    "/media/ebiag/Ebi-4/InfectionExp_Cfel64/InfectionExpCol64.myrmidon"
)

exp_list = [cfel42_exp, cfel1_exp, cfel54_exp, cfel13_exp, cfel55_exp]
# Repeat each element 16 times
exp_list = [x for x in exp_list for _ in range(16)]

In [ ]:
# Start times for experimental phase
exp_start_times = [
    # Cfel 42
    ## Baseline phases
    datetime(2022, 4, 27, 15, 54).astimezone(tz=None),
    datetime(2022, 4, 28, 15, 54).astimezone(tz=None),
    datetime(2022, 4, 29, 15, 54).astimezone(tz=None),
    datetime(2022, 4, 30, 15, 54).astimezone(tz=None),
    datetime(2022, 5, 1, 15, 54).astimezone(tz=None),
    ## Pre/Post
    datetime(2022, 5, 2, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 3, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 4, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 5, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 6, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 7, 9, 0).astimezone(tz=None),
    ## Exp
    datetime(2022, 5, 2, 16, 3).astimezone(tz=None),
    datetime(2022, 5, 3, 15, 53).astimezone(tz=None),
    datetime(2022, 5, 4, 15, 50).astimezone(tz=None),
    datetime(2022, 5, 5, 15, 50).astimezone(tz=None),
    datetime(2022, 5, 6, 15, 55).astimezone(tz=None),
    # Cfel 1
    ## Baseline phases
    datetime(2022, 5, 31, 14, 48).astimezone(tz=None),
    datetime(2022, 6, 1, 14, 48).astimezone(tz=None),
    datetime(2022, 6, 2, 14, 48).astimezone(tz=None),
    # datetime(2022, 6, 3, 14, 48).astimezone(tz=None), # Data corruption on this day
    datetime(2022, 6, 4, 14, 48).astimezone(tz=None),
    ## Pre/Post
    datetime(2022, 6, 5, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 6, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 7, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 8, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 9, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 10, 8, 0).astimezone(tz=None),
    ## Exp
    datetime(2022, 6, 5, 14, 57).astimezone(tz=None),
    datetime(2022, 6, 6, 14, 30).astimezone(tz=None),
    datetime(2022, 6, 7, 14, 49).astimezone(tz=None),
    datetime(2022, 6, 8, 14, 43).astimezone(tz=None),
    datetime(2022, 6, 9, 15, 5).astimezone(tz=None),
    # Cfel 54
    ## Baseline phases
    datetime(2022, 6, 15, 14, 26).astimezone(tz=None),
    datetime(2022, 6, 16, 14, 26).astimezone(tz=None),
    datetime(2022, 6, 17, 14, 26).astimezone(tz=None),
    datetime(2022, 6, 18, 14, 26).astimezone(tz=None),
    datetime(2022, 6, 19, 14, 26).astimezone(tz=None),
    ## Pre/Post
    datetime(2022, 6, 20, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 21, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 22, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 23, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 24, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 25, 8, 0).astimezone(tz=None),
    ## Exp
    datetime(2022, 6, 20, 14, 35).astimezone(tz=None),
    datetime(2022, 6, 21, 14, 21).astimezone(tz=None),
    datetime(2022, 6, 22, 14, 28).astimezone(tz=None),
    datetime(2022, 6, 23, 14, 14).astimezone(tz=None),
    datetime(2022, 6, 24, 14, 31).astimezone(tz=None),
    # Cfel 13
    ## Baseline phases
    datetime(2023, 4, 19, 15, 5).astimezone(tz=None),
    datetime(2023, 4, 20, 15, 5).astimezone(tz=None),
    datetime(2023, 4, 21, 15, 5).astimezone(tz=None),
    datetime(2023, 4, 22, 15, 5).astimezone(tz=None),
    datetime(2023, 4, 23, 15, 5).astimezone(tz=None),
    ## Pre/Post
    datetime(2023, 4, 24, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 25, 7, 30).astimezone(tz=None),
    datetime(2023, 4, 26, 7, 30).astimezone(tz=None),
    datetime(2023, 4, 27, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 28, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 29, 8, 0).astimezone(tz=None),
    ## Exp
    datetime(2023, 4, 24, 15, 29).astimezone(tz=None),
    datetime(2023, 4, 25, 14, 19).astimezone(tz=None),
    datetime(2023, 4, 26, 15, 3).astimezone(tz=None),
    datetime(2023, 4, 27, 16, 43).astimezone(tz=None),
    datetime(2023, 4, 28, 14, 27).astimezone(tz=None),
    # Cfel 55
    ## Baseline phases
    datetime(2023, 4, 14, 14, 40).astimezone(tz=None),
    datetime(2023, 4, 15, 14, 40).astimezone(tz=None),
    datetime(2023, 4, 16, 14, 40).astimezone(tz=None),
    datetime(2023, 4, 17, 14, 40).astimezone(tz=None),
    datetime(2023, 4, 18, 14, 40).astimezone(tz=None),
    ## Pre/Post
    datetime(2023, 4, 20, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 21, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 22, 7, 30).astimezone(tz=None),
    datetime(2023, 4, 23, 7, 30).astimezone(tz=None),
    datetime(2023, 4, 24, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 25, 8, 0).astimezone(tz=None),
    ## Exp
    datetime(2023, 4, 20, 15, 45).astimezone(tz=None),
    datetime(2023, 4, 21, 14, 48).astimezone(tz=None),
    datetime(2023, 4, 22, 14, 17).astimezone(tz=None),
    datetime(2023, 4, 23, 14, 0).astimezone(tz=None),
    datetime(2023, 4, 24, 14, 54).astimezone(tz=None),
    # Cfel 64
    ## Baseline phases
    datetime(2023, 5, 27, 15, 5).astimezone(tz=None),
    datetime(2023, 5, 28, 15, 5).astimezone(tz=None),
    datetime(2023, 5, 29, 15, 5).astimezone(tz=None),
    datetime(2023, 5, 30, 15, 5).astimezone(tz=None),
    datetime(2023, 5, 31, 15, 5).astimezone(tz=None),
    ## Pre/Post
    datetime(2023, 6, 1, 8, 0).astimezone(tz=None),
    datetime(2023, 6, 2, 8, 0).astimezone(tz=None),
    datetime(2023, 6, 3, 8, 0).astimezone(tz=None),
    datetime(2023, 6, 4, 8, 0).astimezone(tz=None),
    datetime(2023, 6, 5, 8, 0).astimezone(tz=None),
    datetime(2023, 6, 6, 8, 0).astimezone(tz=None),
    ## Exp
    datetime(2023, 6, 1, 15, 51).astimezone(tz=None),
    datetime(2023, 6, 2, 14, 44).astimezone(tz=None),
    datetime(2023, 6, 3, 14, 50).astimezone(tz=None),
    datetime(2023, 6, 4, 14, 43).astimezone(tz=None),
    datetime(2023, 6, 5, 14, 52).astimezone(tz=None),
]
exp_end_times = [(start_time + timedelta(hours=6)) for start_time in exp_start_times]

In [15]:
# Remove Base4 from Cfel1 as we don't have data on that day due to data corruption
# It is the at index 19
del colony_list[19]
del exp_list[19]
del phase_list[19]

In [16]:
interact_df_list = [
    interaction_times(exp, start_time, end_time, colony, phase)
    for exp, start_time, end_time, colony, phase in zip(
        exp_list, exp_start_times, exp_end_times, colony_list, phase_list
    )
]

Computing ant interactions: 100%|███▉| 359/360 [00:05<00:00, 70.02tracked min/s]


In [6]:
# Colony Cfel64
cfel64_exp = fm.Experiment.Open(
    "/media/egeorge/Ebi-4/InfectionExp_Cfel64/InfectionExpCol64.myrmidon"
)
colony = "Cfel64"
phase_list = [
    "Base1",
    "Base2",
    "Base3",
    "Base4",
    "Base5",
    "PostC",
    "PostR1",
    "PostR2",
    "PostR3",
    "PostR4",
    "PostR5",
    "R1",
    "R2",
    "R3",
    "R4",
    "R5",
]
exp_start_times = [
    # Cfel 64
    ## Baseline phases
    datetime(2023, 5, 27, 15, 5).astimezone(tz=None),
    datetime(2023, 5, 28, 15, 5).astimezone(tz=None),
    datetime(2023, 5, 29, 15, 5).astimezone(tz=None),
    datetime(2023, 5, 30, 15, 5).astimezone(tz=None),
    datetime(2023, 5, 31, 15, 5).astimezone(tz=None),
    ## Pre/Post
    datetime(2023, 6, 1, 8, 0).astimezone(tz=None),
    datetime(2023, 6, 2, 8, 0).astimezone(tz=None),
    datetime(2023, 6, 3, 8, 0).astimezone(tz=None),
    datetime(2023, 6, 4, 8, 0).astimezone(tz=None),
    datetime(2023, 6, 5, 8, 0).astimezone(tz=None),
    datetime(2023, 6, 6, 8, 0).astimezone(tz=None),
    ## Exp
    datetime(2023, 6, 1, 15, 51).astimezone(tz=None),
    datetime(2023, 6, 2, 14, 44).astimezone(tz=None),
    datetime(2023, 6, 3, 14, 50).astimezone(tz=None),
    datetime(2023, 6, 4, 14, 43).astimezone(tz=None),
    datetime(2023, 6, 5, 14, 52).astimezone(tz=None),
]
exp_end_times = [(start_time + timedelta(hours=6)) for start_time in exp_start_times]


In [7]:
[
    interaction_times(cfel64_exp, start_time, end_time, colony, phase)
    for start_time, end_time, phase in zip(exp_start_times, exp_end_times, phase_list)
]

Computing ant interactions:   0%|              | 0/360 [00:06<?, ?tracked min/s]


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [ ]:
f_myrmidon = "/media/egeorge/Redhood/Experiment3/woundcare_cfell54_T3.myrmidon"
exp = fm.Experiment.Open(f_myrmidon)

In [ ]:
# output_focal_interactions([106,22], (2022, 5, 2, 16, 3), 6)
focal_ants = [[108, 107], [114, 9], [62, 87], [12, 83], [53, 101]]
start_times = [
    (2022, 6, 20, 14, 35),
    (2022, 6, 21, 14, 21),
    (2022, 6, 22, 14, 28),
    (2022, 6, 23, 14, 14),
    (2022, 6, 24, 14, 31),
]

In [ ]:
[output_focal_interactions(x, y, 6) for x, y in zip(exp, focal_ants, start_times)]